In [ ]:
import requests

# store login data in login.py
%run login.py

# login query as multiline formatted string
# this assumes that login and pwd are defined 
# above

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""


url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']

# Creating Rectangular Zones

To analyze dwell time, traffic, and other statistics of the areas, we will divide the areas into rectangular zones. The zones are created and passed into queries.

In [ ]:
import json
import numpy as np
from spec_constants import *

zone_ids = {scene:np.zeros((ROWS, COLUMNS)) for scene in SCENES}

# There are three cameras tracking three areas
for camera_number in range(NUM_CAMERAS):
    # Get the coordinates of a rectangular region for each pair of i and j. The top left pixel is (0, 0) and the 
    # bottom right pixel is (640, 480).
    for i in range(ROWS):
        for j in range(COLUMNS):
            # This query creates a behavior zone for the rectangular area above whose traffic is to be analyzed
            get_behavior_zone_query = """
            mutation {
              createBehaviorZone(
                serialno: "SWLSANDBOX""" + str(camera_number + 1) + """",
                demarcation: """ + str(ZONE_COORDINATES[CAMERA_TO_SCENE[camera_number]][i * COLUMNS + j]) + """,
                text: "prea3-area""" + str(camera_number + 1) + "-" + str(ROWS) + "-by-" + str(COLUMNS) + "-row" + str(i + 1) + "-column" + str(j + 1) + """") {
                        behaviorZone {
                            rawId
                        }
                }
            }
            """
            request = requests.post(url, json={'query': get_behavior_zone_query}, headers = {'Authorization':token})
            # Add the zone and its coordinates to the list of zones associated with the camera number
            zone_ids[CAMERA_TO_SCENE[camera_number]][i, j] = json.dumps(
                request.json()['data']['createBehaviorZone']['behaviorZone']['rawId'], indent = 1)
            print(f"Created zone for {CAMERA_TO_SCENE[camera_number]} in row {i}, column {j}", end="\r")

In [ ]:
import numpy
import pandas as pd

# Write the zones IDs into csv files, one for each area
for camera_number in range(NUM_CAMERAS):
    numpy.savetxt(
        "./Zehao-files/q1_zones/" + str(ROWS) + "x" + str(COLUMNS) + " " + CAMERA_TO_SCENE[camera_number] + ".csv"
                  , zone_ids[CAMERA_TO_SCENE[camera_number]], delimiter=",")

# Creating Area Zones

The zone dwell time query provides many metrics that are not provided by the feed queries so we will get data for an entire area by creating a zone representing it.

In [ ]:
import json
import numpy as np
from spec_constants import *

area_zone_id = []

# Create a zone for each of the three areas
for camera_number in range(NUM_CAMERAS):
    get_area_zone_query = """
    mutation {
      createBehaviorZone(
        serialno:"SWLSANDBOX""" + str(camera_number + 1) + """",
        demarcation:[[0, 0], [640, 0], [640, 480], [0, 480]],
        text: "area""" + str(camera_number + 1) + """") {
                behaviorZone {
                    rawId
                }
        }
    }
    """
    request = requests.post(url, json={'query': get_area_zone_query}, headers = {'Authorization':token})
    # Add the zone and its coordinates to the list of zones associated with the camera number
    area_zone_id.append(json.dumps(
        request.json()['data']['createBehaviorZone']['behaviorZone']['rawId'], indent = 1))